# Install conda

In [1]:
import sys
! wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
! chmod +x Anaconda3-2020.02-Linux-x86_64.sh
! bash ./Anaconda3-2020.02-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda update -n base -c defaults conda -y
! conda config --add channels bioconda
! conda config --add channels conda-forge
! conda install biopython -y

--2020-07-25 02:52:48--  https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 546910666 (522M) [application/x-sh]
Saving to: ‘Anaconda3-2020.02-Linux-x86_64.sh’

Anaconda3-2020.02-L 100%[===================>] 521.57M   201MB/s    in 2.6s    

2020-07-25 02:52:51 (201 MB/s) - ‘Anaconda3-2020.02-Linux-x86_64.sh’ saved [546910666/546910666]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.12=py37_0
    - anacon

# Create and set up virtual environment for BiG-SCAPE

In [9]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

# Create virtual environment for BiG-SCAPE, then install dependencies, BiG-SCAPE, and databases into it (this will take a while)
conda create --prefix /usr/local/envs/bigscape python==3.6 -y
conda install --name bigscape hmmer biopython mafft fasttree networkx numpy scipy scikit-learn=0.19.1 -y
conda activate bigscape

# Clone BiG-SCAPE from Git and install in virtual environment
cd /usr/local/envs/bigscape
git clone https://git.wur.nl/medema-group/BiG-SCAPE.git

# Download Pfam database and check that everything was installed properly
cd BiG-SCAPE
mkdir -p databases
cd databases
wget ftp://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam32.0/Pfam-A.hmm.gz && gunzip Pfam-A.hmm.gz 
hmmpress Pfam-A.hmm

# Check that everything was installed correctly
cd ..
python bigscape.py --version
conda deactivate

Solving environment: | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / done

## Package Plan ##

  environment location: /usr/local/envs/bigscape

  added / updated specs:
    - python==3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ncurses-5.9                |               10         1.1 MB  conda-forge
    openssl-1.0.2u             |       h516909a_0         3.2 MB  conda-forge
    python-3.6.0               |                2        18.9 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    readline-6.2               |                0         713 KB  conda-forge
    setuptools-49.2.0          |   py36h9f0ad1d_0         910 KB  conda-forge
    sqlite-3.13.0              |

# Run BiG-SCAPE on sample files

In [10]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

# Download example dataset
cd /usr/local/envs/bigscape/BiG-SCAPE
mkdir -p demo
cd demo
wget https://raw.githubusercontent.com/nselem/bigscape-corason/master/scripts/data_bigscape_corason.sh
chmod a+x data_bigscape_corason.sh
bash data_bigscape_corason.sh -b

# Run BiG-SCAPE on example dataset
conda activate bigscape

python /usr/local/envs/bigscape/BiG-SCAPE/bigscape.py \
  --inputdir /usr/local/envs/bigscape/BiG-SCAPE/demo/gbks \
  --outputdir /usr/local/envs/bigscape/BiG-SCAPE/demo/output/BiG-SCAPE \
  --pfam_dir /usr/local/envs/bigscape/BiG-SCAPE/databases

conda deactivate

--2020-07-25 03:20:42--  https://raw.githubusercontent.com/nselem/bigscape-corason/master/scripts/data_bigscape_corason.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343 [text/plain]
Saving to: ‘data_bigscape_corason.sh’

data_bigscape_coras 100%[===================>]     343  --.-KB/s    in 0s      

2020-07-25 03:20:42 (14.7 MB/s) - ‘data_bigscape_corason.sh’ saved [343/343]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  877k  100  877k    0     0  1317k      0 --:--:-- --:--:-- --:--:-- 1315k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left

# Run BiG-SCAPE on several genomes

Often when running BiG-SCAPE in Google Colab, you'll have your .gbk files stored in your Google Drive. The problem with this approach is that Google Drive path names contain spaces, which are not allowed in BiG-SCAPE input or output. The approach below is a workaround in which shortcuts are made from the desired directory to a directory in the Colab virtual machine, then BiG-SCAPE is run and the outputs stored in the VM, and finally they are copied to the desired output directory in Google Drive.

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

# Specify input and output directories
input_dir="/gdrive/My Drive/BiGSCAPE"
output_dir="/gdrive/My Drive/BiGSCAPE/output"

# Create symlinks for input and output directories in Google Drive because path of BiG-SCAPE input directory can't contain spaces
input_dir_linked="/usr/local/envs/bigscape/BiG-SCAPE/input"
output_dir_linked="/usr/local/envs/bigscape/BiG-SCAPE/output"
rm -r "$output_dir_linked"
mkdir -p "$input_dir_linked"
mkdir -p "$output_dir_linked"
ln -sf "$input_dir" "$input_dir_linked"
ln -sf "$output_dir" "$output_dir_linked"

# Run BiG-SCAPE on example dataset
conda activate bigscape

python /usr/local/envs/bigscape/BiG-SCAPE/bigscape.py \
  --inputdir "$input_dir_linked"/* \
  --outputdir "$output_dir_linked" \
  --pfam_dir /usr/local/envs/bigscape/BiG-SCAPE/databases \
  --cutoffs 0.3 0.4 0.5

conda deactivate

# Copy output from linked output dir to desired output dir
mkdir -p "$output_dir"
cp -r "$output_dir_linked" --target-directory="$output_dir"